In [9]:
import cv2
import json
import os
import numpy as np
from sklearn.cluster import KMeans

In [3]:
dataset_path = "./dataset/dataset-1"
dataset_folder = os.listdir(dataset_path)

anno_file_name = dataset_folder[1]

with open(f"{dataset_path}/" + anno_file_name, "r") as file:
    anno_file_content = json.load(file)

# print(anno_file_content["categories"])

# class map is a mapping that maps class to a unique number
class_map = {}
for category_info in anno_file_content["categories"]:
    print(category_info["name"])
    class_map[category_info["id"]] = category_info["name"]

print(class_map)

mint
healthy_apple_mint
healthy_chocolate_mint
healthy_pepper_mint
healthy_pineapple_mint
non-mint
non_mint
{0: 'mint', 1: 'healthy_apple_mint', 2: 'healthy_chocolate_mint', 3: 'healthy_pepper_mint', 4: 'healthy_pineapple_mint', 5: 'non-mint', 6: 'non_mint'}


In [4]:
print(anno_file_content.keys())

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])


In [5]:
def show_image(image, message):
    cv2.imshow("first image", image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [6]:
# image address map
# will map the address of an image to a unique number
image_address_map = {}

for image_info in anno_file_content["images"]:
    image_path = dataset_path + "/images/" + image_info["file_name"]
    image_address_map[image_info["id"]] = image_path


In [ ]:
# class map
# image address map


# every annotation in the annotations list is an object( in our case a leaf ), and it has info about what image 
# the object came from and what is its class
# the image address map will be used here to get the address of the image from image_id and fetch the image
# class_id will be used to give label to the object( leaf ) detect in the feature vector
# segmentations variable stores the x and y coordinate of points on the polygon that surrounds the object



for object_annotation in anno_file_content["annotations"]:
    image_address_id = object_annotation["image_id"]
    segmentations = object_annotation["segmentation"]
    class_id = object_annotation["category_id"]
    image_address = image_address_map[image_address_id]

    image = cv2.imread(image_address)

    show_image(image, "image")

    # now we try to segment the object from the rest of the image using the point stored in segmentations
    
    points = np.array(segmentations, dtype=np.int32).reshape(-1, 2)

    # initial_mask is the mask on the polygon 
    initial_mask = np.zeros(image.shape[:2], dtype=np.uint8)


    # fillPoly will fill the inside of the polygon with white color
    cv2.fillPoly(initial_mask, [points], 255)

    # anno_object stores the annotation polygon superimposed on the image
    anno_object = cv2.bitwise_and(image, image, mask=initial_mask)

    
    # preprocessing on the anno_object to isolate the leaf part
    

    break

